In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        self.upper_layer1 = nn.Sequential(
            nn.Conv2d(4,64,7),
            nn.ReLU(),
            nn.MaxPool2d((4,4),stride = 1)
        )
        self.upper_layer2 = nn.Sequential(
            nn.Conv2d(64,64,3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride = 1)
        )
        self.under_layer1 = nn.Sequential(
            nn.Conv2d(4,160,13),
            nn.ReLU()
        )
        self.final_layer = nn.Conv2d(224,5,21)
        
    def forward(self, x):
        upper_x = self.upper_layer2(self.upper_layer1(x))
        under_x = self.under_layer1(x)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv1(x)), (4, 4),stride = 1)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv2(upper_x)), (2, 2), stride = 1)
        #under_x = F.relu(self.under_conv1(x))
        final_x = torch.cat((under_x, upper_x), 1)
        out = self.final_layer(final_x)
        return out
        
net = TwoPathConv()
print(net)

x = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = net.forward(x)
print(y_pred)
#y_pred = y_pred.data.resize_(1,5)
#y_pred = Variable(y_pred,requires_grad = True)
#print(y_pred.size())
#print(y_pred.view)

TwoPathConv (
  (upper_layer1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(4, 4), stride=(1, 1), dilation=(1, 1))
  )
  (upper_layer2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(1, 1), dilation=(1, 1))
  )
  (under_layer1): Sequential (
    (0): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
    (1): ReLU ()
  )
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)
Variable containing:
(0 ,0 ,.,.) = 
 -0.0328

(0 ,1 ,.,.) = 
  0.4779

(0 ,2 ,.,.) = 
 -0.0311

(0 ,3 ,.,.) = 
  0.3174

(0 ,4 ,.,.) = 
  0.0089
[torch.FloatTensor of size 1x5x1x1]



In [2]:
import h5py
f5 = h5py.File('training.h5','r')
import time
#get the training set for phase 1
f=open("trainval-balanced.txt", "r")
content=f.readlines()
X_train_phase1=[]
y_train_phase1=[]
i=0
from random import shuffle
shuffle(content)
prev_time=time.clock()
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    case, x, y, z, l = item[0], int(item[1]), int(item[2]), int(item[3]), int(item[4])
    case1 = case[:2]
    case2 = case[3:]
    X_train_phase1.append(f5[case1][case2][:, x-16:x+17, y-16:y+17, z])
    y_train_phase1.append(l)
    i+=1
    if i%1000==0:
        print("phase1: "+str(i*100.0/len(content))+"%")
        print("time: "+str(time.clock()-prev_time))
        print("averaged time: "+str((time.clock()-prev_time)/i))
    if i>10000:
        break
f.close()
print ("phase 1 data preparation process completed.")

#get the training set for phase 2 and the validation set
data_val=[]
f_in=open("trainval.txt", "r")
content=f_in.readlines()
from random import shuffle
shuffle(content)
X_train_phase2=[]
y_train_phase2=[]
X_val=[]
y_val=[]
i=0
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    case, x, y, z, l = item[0], int(item[1]), int(item[2]), int(item[3]), int(item[4])
    case1 = case[:2]
    case2 = case[3:]
    i += 1
    if i%3000==0:
        print("phase1: "+str(i*100.0/len(content))+"%")
        print("time: "+str(time.clock()-prev_time))
        print("averaged time: "+str((time.clock()-prev_time)/i))
    if i%3000==0:
        X_val.append(f5[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_val.append(l)
    else:
        X_train_phase2.append(f5[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_train_phase2.append(l)
    if i>30000:
        break
    
f_in.close()
print ("phase 2 data preparation process completed.")

f5.close()


phase1: 0.011113215213191476%
time: 2.6491849999999992
averaged time: 0.0026505519999999974
phase1: 0.02222643042638295%
time: 4.670807
averaged time: 0.0023355005
phase1: 0.03333964563957443%
time: 6.717321999999999
averaged time: 0.0022391986666666667
phase1: 0.0444528608527659%
time: 8.755136
averaged time: 0.002188927249999999
phase1: 0.05556607606595738%
time: 10.813628999999999
averaged time: 0.0021628248
phase1: 0.06667929127914886%
time: 12.846536
averaged time: 0.0021411645
phase1: 0.07779250649234033%
time: 14.750248
averaged time: 0.0021072561428571433
phase1: 0.0889057217055318%
time: 16.643996
averaged time: 0.002080573
phase1: 0.10001893691872328%
time: 18.674301
averaged time: 0.0020749549999999994
phase1: 0.11113215213191475%
time: 20.719751000000002
averaged time: 0.0020720135
phase 1 data preparation process completed.
phase1: 0.012107174158512047%
time: 77.414353
averaged time: 0.025804980666666665
phase1: 0.024214348317024095%
time: 83.296902
averaged time: 0.013882

In [3]:
import numpy as np
print (len(X_train_phase1))
print (len(y_train_phase1))
print (len(X_train_phase2))
print (len(y_train_phase2))
print (len(X_val))
print (len(y_val))
X_train_phase1 = np.array(X_train_phase1)
y_train_phase1 = np.array(y_train_phase1)
X_train_phase2 = np.array(X_train_phase2)
y_train_phase2 = np.array(y_train_phase2)
X_val = np.array(X_val)
y_val = np.array(y_val)


10001
10001
29991
29991
10
10


In [4]:
print (torch.cuda.is_available())

True


In [5]:
import torch.nn.init as ninit
def init_net(net):
    for param in net.parameters():
        ninit.uniform(param.data, a=-5e-3, b=5e-3)

In [ ]:
def create_batch_phase1(index, batch_size):
    start=index*batch_size
    end=start+batch_size
    X_batch= X_train_phase1[start:end]
    y_batch= y_train_phase1[start:end]
    X_batch = torch.from_numpy(X_batch)
    y_batch = torch.from_numpy(y_batch)
    return X_batch, y_batch

def create_batch_phase2(index, batch_size):
    start=index*batch_size
    end=start+batch_size
    X_batch= X_train_phase2[start:end]
    y_batch= y_train_phase2[start:end]
    X_batch = torch.from_numpy(X_batch)
    y_batch = torch.from_numpy(y_batch)
    return X_batch, y_batch

In [ ]:
import time
import numpy as np
num_train=len(X_train_phase1)
num_val=len(X_val)
batch_size=900
val_size=num_val
num_epoch=2.0
num_times=int(float(num_train)/batch_size*num_epoch)
print (num_times)
learning_rate = 5e-3
reg=5e-5
net = TwoPathConv()
print(net)
init_net(net)
net.cuda(0)

#create validation set
X_val = torch.from_numpy(np.array(X_val))
y_val = torch.from_numpy(np.array(y_val))
X_val = Variable(X_val.cuda(0), requires_grad=False)

prev_time = time.clock()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=reg)
net.zero_grad()
for i in range(21):  # loop over the dataset multiple times
    X_batch=None
    X_batch, y_batch = create_batch_phase1(i, batch_size)
    X_batch, y_batch = Variable(X_batch.cuda(0)), Variable(y_batch.cuda(0), requires_grad = False)
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    net.zero_grad()
    if i % 5 == 0:
        print ("")
        print ('time used %.3f' % (time.clock()-prev_time))
        print (str(float(i)/num_times*100)+"% completed")
        print (loss)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda(0)).sum()
        print('Validation accuracy:', float(correct)/val_size)
print ("phase1 successfully trained!")
#torch.save(net.state_dict(), "premature_net.txt")
print ("phase1 successfully saved!")

22
TwoPathConv (
  (upper_layer1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(4, 4), stride=(1, 1), dilation=(1, 1))
  )
  (upper_layer2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(1, 1), dilation=(1, 1))
  )
  (under_layer1): Sequential (
    (0): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
    (1): ReLU ()
  )
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)

time used 0.548
0.0% completed
Variable containing:
 1.6142
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Validation accuracy: 0.0

time used 1.982
22.727272727272727% completed
Variable containing:
 1.3925
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Validation accuracy: 0.1

time used 3.412
45.45454545454545% completed


In [ ]:
def adam(x, dx, config = None):
    if config is None: 
        config = {}
        config.setdefault('learning_rate',1e-4)
        config.setdefault('beta1',0.9)
        config.setdefault('beta2',0.999)
        config.setdefault('epsilon',1e-8)
        config.setdefault('m',torch.FloatTensor(x.size()).zero_().cuda(0))
        config.setdefault('v',torch.FloatTensor(x.size()).zero_().cuda(0))
        config.setdefault('t',0)
    
    next_x = None
    config['t'] += 1
    x.cuda(0)
    dx.cuda(0)
    config['m'] = config['beta1'] * config['m'] + (1 - config['beta1']) * dx
    config['v'] = config['beta2'] * config['v'] + (1 - config['beta2']) * dx**2
    mb = config['m'] / (1 - config['beta1'] ** config['t'])
    vb = config['v'] / (1 - config['beta2'] ** config['t'])
    next_x = x - config['learning_rate'] * mb / (np.sqrt(vb) + config['epsilon'])
    return next_x, config

def sgd_momentum(w, dw, config = None):

    if config is None: 
        config = {}
        config.setdefault('learning_rate', 5e-4)
        config.setdefault('momentum', 0.9)

    v = config.get('velocity', torch.FloatTensor(w.size()).zero_().cuda(0))
    next_w = None

    v = config['momentum'] * v - config['learning_rate'] * dw
    next_w = w + v
    config['velocity'] = v

    return next_w, config


In [ ]:
import time
import numpy as np
num_train=len(data_train_phase2)
num_val=len(data_val)
batch_size=900
val_size=num_val
num_epoch=2.0
num_times=int(float(num_train)/batch_size*num_epoch)
print (num_times)
learning_rate = 5e-4
reg=5e-5
net.load_state_dict(torch.load("premature_net.txt"))
net.cuda(0)

prev_time = time.clock()
config_weight = None
config_bias = None
net.zero_grad()
for i in xrange(21):  # loop over the dataset multiple times
    X_batch = None
    batch_time=time.clock()
    X_batch, y_batch = create_batch_phase2(i, batch_size)
    print("batch_time: "+ str(time.clock()-batch_time))
    X_batch, y_batch = Variable(X_batch.cuda(0)), Variable(y_batch.cuda(0), requires_grad = False)
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    dweight = net.final_layer.weight.grad.data+reg*net.final_layer.weight.data
    dbias = net.final_layer.bias.grad.data
    net.final_layer.weight.data, config_weight = sgd_momentum(net.final_layer.weight.data, dweight, config_weight)
    net.final_layer.bias.data, config_bias = sgd_momentum(net.final_layer.bias.data, dbias, config_bias)
    net.zero_grad()
    if i % 5 == 0:
        print ("")
        print ('time used %.3f' % (time.clock()-prev_time))
        print (str(float(i)/num_times*100)+"% completed")
        print (loss)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda(0)).sum()
        print('Validation accuracy:', float(correct)/val_size)
print ("phase2 successfully trained!")
#torch.save(net.state_dict(), "premature_net_phase2_without_biasreg.txt")
print ("phase2 successfully saved!")